In [ ]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import time
import pickle

In [ ]:
from PIL import Image
# from pillow import Image

import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model

In [ ]:
model = keras.applications.VGG16(weights='imagenet', include_top=True)
print("model loaded")
model.summary()

In [ ]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [ ]:
# set up the feature extractor

feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
print("feature extractor setup")
feat_extractor.summary()

### Analyse samples and save csv

In [ ]:
current_path = "/home/rte/data/images/random/100k/classification/"
# current_path = convert_path

image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 100000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(current_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
num_x = len(images)
print("keeping %d images to analyze" % num_x)

In [ ]:
start = time.time()

all_predictions = []

features = []

writefile = "predictions_100k.txt"
f = open(writefile, "a+")
        
for i, image_path in enumerate(images):
    if i % 500 == 0:
        end = time.time()
        elap = end - start;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        start = time.time()
    img, x = load_image(image_path)
    
    f.write(os.path.basename(image_path))

#     feat = feat_extractor.predict(x)[0]
#     features.append(feat)
    
    predictions = model.predict(x)
    
#     print(predictions)
#     print(decode_predictions(predictions))
    
    for _, pred, prob in decode_predictions(predictions)[0]:
#         print("predicted %s with probability %0.3f" % (pred, prob))
        f.write("," + str(pred) + "," + str(prob))
                
    f.write("\n")
        
print('finished predicting class for %d images' % len(images))

f.close()



In [ ]:
# write images, predictions to a pickle file

f = "classification_vgg_subset" + ".pickle"

print(f)

# WRITE
with open(f, "wb") as write_file:
    pickle.dump([images, features], write_file)
    write_file.close()
